In [1]:
# created on Dec 24, 2020
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

!python -m pip install tweepy
import tweepy, json, time

In [2]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.f = open(file, 'a', encoding="utf-8")
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                # {'limit': {'track': 13, 'timestamp_ms': '1585851016736'}}

                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    record = '%s, %s, %f, %f, %s \n' % (username, created_at, lng, lat, text)
                    print(record)
                    self.f.write(record)
                else:
                    pass
        else:
            self.f.close()
            print("finished")
            return False

In [15]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = "assets/tweets.csv"

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "x8TUQ5oL3UM8DzDwFiZikTUDg"
    consumer_secret = "1QyAiFA30cTWUGLutakUNQjhKixjjc3S0aHPLNj5yuxgNxJdV7"
    access_token = "3324993781-GXyug0tDfnBVZ20iolQ3ca5y3PwWsOCjc29DagI"
    access_token_secret = "Xu1YePrjUH1X7N8PRsxwOA50EhjEJAYAhCUQHKUNf4LO3"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-24.009291, 15.806093, 10.527828, 39.888123,  # Contiguous US 
                 -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
                 -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii 

    stream_listener = StreamListener(time_limit=70, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(track=['election'], is_async=True)

{'created_at': 'Wed Jan 27 19:40:21 +0000 2021', 'id': 1354514603187830790, 'id_str': '1354514603187830790', 'text': 'RT @NBCNews: Goya’s board of directors reportedly votes to censure CEO Robert Unanue after he made unfounded public claims about voter frau…', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1367563170, 'id_str': '1367563170', 'name': 'Robby T〽️🐺🦅🌬❄️✨🅰️🐘🏈‼️', 'screen_name': 'rezzhead', 'location': 'EBCI Reservation, NC REZTOWN', 'url': None, 'description': "Onkwehonwe. {Beyond ideas of wrongdoing and rightdoing there is a field. I'll meet you there - Rumi} #RollTide 🅰️🐘💯❤️UofA Alumnus. 🏈🏀⚾️🥎🏸⛳️Fan. #Resist🌊", 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 2057, 'friends_count': 1148, 'listed_c

{'created_at': 'Wed Jan 27 19:40:21 +0000 2021', 'id': 1354514604261576704, 'id_str': '1354514604261576704', 'text': "RT @TheGatorGamer: So now it's okay to take out political vendettas from the previous election cycle? Okay. When the right seizes power aga…", 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1069044141856878594, 'id_str': '1069044141856878594', 'name': 'Savoyard Reaction', 'screen_name': 'SavoCRx', 'location': None, 'url': None, 'description': None, 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 120, 'friends_count': 449, 'listed_count': 1, 'favourites_count': 33689, 'statuses_count': 4183, 'created_at': 'Sun Dec 02 01:42:40 +0000 2018', 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'lang'

{'created_at': 'Wed Jan 27 19:40:22 +0000 2021', 'id': 1354514606010494978, 'id_str': '1354514606010494978', 'text': 'RT @jennycohn1: Targeting African Americans in 2016, white nationalist encouraged these voters to “vote” via text. Text voting doesn’t exis…', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1324396565071454208, 'id_str': '1324396565071454208', 'name': 'SlBailey', 'screen_name': 'Arizonaajgma', 'location': 'Arizona, USA', 'url': None, 'description': 'i believe that we are our brothers keepers. i am a proud independent voter.  I chose democracy over fascism. equality for all. Black lives http://matter.No DMs', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 2396, 'friends_count': 2254, 'listed

{'created_at': 'Wed Jan 27 19:40:22 +0000 2021', 'id': 1354514606912196609, 'id_str': '1354514606912196609', 'text': "My pinned tweet from Jan 14th. Let's see how this develops.", 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1081643357607096320, 'id_str': '1081643357607096320', 'name': 'Traderhank', 'screen_name': 'Traderhankhill', 'location': 'Calgary, Alberta', 'url': None, 'description': 'StOnKs. Not investment advice, I tell you hwat. P.Eng, M.Eng Chemical Engineer. 🇨🇦', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 290, 'friends_count': 110, 'listed_count': 6, 'favourites_count': 1736, 'statuses_count': 2849, 'created_at': 'Sat Jan 05 20:07:27 +0000 2019', 'utc_offset': None, 'time_zone': None, 'geo_enabled': F

{'created_at': 'Wed Jan 27 19:40:23 +0000 2021', 'id': 1354514608606900224, 'id_str': '1354514608606900224', 'text': 'RT @Public_Citizen: After the Capitol Hill insurrection, Big Oil rushed to distance itself from the chaos. \n\nBut it’s too late. \n\nFossil fu…', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 3245379856, 'id_str': '3245379856', 'name': 'jash', 'screen_name': 'HungryMerican', 'location': 'Chicago, IL', 'url': 'http://instagram.com/humgry_art', 'description': 'Personal stuff and political stuff. Retweets are not endorsements.', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 59, 'friends_count': 54, 'listed_count': 4, 'favourites_count': 33645, 'statuses_count': 5722, 'created_at': 'Sun May 

...

{'created_at': 'Wed Jan 27 19:40:59 +0000 2021', 'id': 1354514759945756677, 'id_str': '1354514759945756677', 'text': 'RT @SunSentinel: Delray Beach man accused of using social-media accounts to intimidate voters during 2016 election https://t.co/18YV0EMBas…', 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 51330975, 'id_str': '51330975', 'name': 'RIP Trayvon Martin', 'screen_name': 'dierdrelewis', 'location': None, 'url': None, 'description': None, 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 6463, 'friends_count': 7224, 'listed_count': 284, 'favourites_count': 7116, 'statuses_count': 290731, 'created_at': 'Sat Jun 27 03:24:34 +0000 2009', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'cont

{'created_at': 'Wed Jan 27 19:40:59 +0000 2021', 'id': 1354514761304715265, 'id_str': '1354514761304715265', 'text': 'RT @rmayemsinger: @LeaderMcConnell You mean like stealing a SCOTUS seat? You mean like forcing an unqualified religious zealot into SCOTUS…', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 391663063, 'id_str': '391663063', 'name': 'Monica Glambeau', 'screen_name': 'EarlyToRise31', 'location': 'United States', 'url': None, 'description': 'A new day has dawn. Do good. Be good. Treat others as you would like to be treated. #BLM. ✊🏽', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 1242, 'friends_count': 877, 'listed_count': 15, 'favourites_count': 85524, 'statuses_count': 58606, 'created_at': '

{'created_at': 'Wed Jan 27 19:40:59 +0000 2021', 'id': 1354514761975738381, 'id_str': '1354514761975738381', 'text': 'RT @xctlot: Straightforward soviet prosecution of political dissidents', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1340736735974703108, 'id_str': '1340736735974703108', 'name': 'AF volus', 'screen_name': 'AF_volus', 'location': None, 'url': None, 'description': 'Cozy • lurker • AF', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 17, 'friends_count': 112, 'listed_count': 0, 'favourites_count': 6911, 'statuses_count': 111, 'created_at': 'Sun Dec 20 19:12:30 +0000 2020', 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'lang': None, 'contributors_enabled': False, 'is_translato

{'created_at': 'Wed Jan 27 19:40:59 +0000 2021', 'id': 1354514763322109952, 'id_str': '1354514763322109952', 'text': 'RT @JovanHPulitzer: #Scan154MillionBallots Asked many times "are u willing to do a complete forensic audit on every 2020 General Election B…', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1283434271219355648, 'id_str': '1283434271219355648', 'name': 'Dr. Kevin McCallister', 'screen_name': 'KMMofc', 'location': None, 'url': None, 'description': 'Here for Music and Politics', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 17, 'friends_count': 232, 'listed_count': 0, 'favourites_count': 2094, 'statuses_count': 4557, 'created_at': 'Wed Jul 15 16:12:46 +0000 2020', 'utc_offset': None, 'time_zone

{'created_at': 'Wed Jan 27 19:41:00 +0000 2021', 'id': 1354514764488208387, 'id_str': '1354514764488208387', 'text': 'RT @RobbersonJon: Folks please hear me: \n\nTHERE IS NO “NEXT ELECTION” \n\nWe just sent the man who received 75M legit votes home \n\nWe watched…', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 434573147, 'id_str': '434573147', 'name': "Citizens' Dawn ✝️", 'screen_name': 'CitizensDawn', 'location': 'Boston, MA', 'url': 'https://www.citizensdawn.org', 'description': "Follow Citizens' Dawn at http://gab.ai/CitizensDawn . Bitchute: http://bitchute.com/channel/citize…", 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 142, 'friends_count': 806, 'listed_count': 0, 'favourites_count': 4941, 'stat